In [1]:
import os
import torch
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.embeddings import SentenceTransformerEmbeddings


/home/fermikira/anaconda3/envs/llms_env/lib/python3.12/site-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [2]:
llm = ChatGroq(groq_api_key=os.environ['GROQ_API_KEY'], model_name="llama3-70b-8192")

In [3]:
loader = TextLoader('GuionCHO.txt')
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [5]:
spanish_embeddings = SentenceTransformerEmbeddings(model_name="")

long_es_embeddings = SentenceTransformerEmbeddings(model_name="jinaai/jina-embeddings-v2-base-es")

multil = SentenceTransformerEmbeddings(model_name="sentence-transformers/distiluse-base-multilingual-cased-v1")

/tmp/ipykernel_215779/1592203801.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  spanish_embeddings = SentenceTransformerEmbeddings(model_name="jinaai/jina-embeddings-v2-base-es")
Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-base-es and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense

In [6]:
persist_directory = 'db_2000_0_espa_jina'

## Here is the nmew embeddings being used
#embedding = instructor_embeddings
embedding = spanish_embeddings
vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


In [7]:
len(vectordb)

127

In [14]:
retriever = vectordb.as_retriever(search_kwargs={"k":20})


In [15]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [16]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.page_content)

In [19]:
# full example
query = "CUAL ES LA SEDE MEDELLIN?"
llm_response = qa_chain.invoke(query)
process_llm_response(llm_response)

La sede de Medellín es:

* Medellín: Carrera 43a N° 23-25 local 128 Centro Comercial Avenida Mall.
Horario: Lunes a Viernes 8:00 a.m. a 5:30 p.m. 
Sábados 8:00 a.m. a 01:00 p.m.


Sources:
PROBLEMATICAS / ERRORES FRECUENTES / CAIDAS DEL SISTEMA / FALLAS EN EL SISTEMA/ FALLAS EN EL SISTEMAA INICIO DE MES

En este momento estamos presentando una actualización general, la cual tomará un tiempo aproximado de dos horas. Si el problema persiste, puede comunicarse a través de nuestra línea de atención al cliente (601) 749 9000.

OTROS / ERROR CUANDO REALIZABA UNA TRANSACCIÓN / ERROR
De acuerdo a su consulta, esta solicitud se debe realizar por medio del correo electrónico de servicioalcliente@finanzauto.com.co adjuntando la siguiente documentación:

* Carta donde nos informe el trámite a solicitar, es necesario indicar el motivo y los cambios deseados.
* La solicitud debe tener firma, huella, número de cédula y número de celular legibles. 
* Fotocopia de la tarjeta de propiedad por ambas cara

In [12]:
sum([len(llm_response['source_documents'][i].page_content) for i in range(len(llm_response['source_documents']))])

6387

In [13]:
llm_response['source_documents']

[Document(metadata={'source': 'GuionCHO.txt'}, page_content='PROBLEMATICAS / ERRORES FRECUENTES / CAIDAS DEL SISTEMA / FALLAS EN EL SISTEMA/ FALLAS EN EL SISTEMAA INICIO DE MES\n\nEn este momento estamos presentando una actualización general, la cual tomará un tiempo aproximado de dos horas. Si el problema persiste, puede comunicarse a través de nuestra línea de atención al cliente (601) 749 9000.\n\nOTROS / ERROR CUANDO REALIZABA UNA TRANSACCIÓN / ERROR'),
 Document(metadata={'source': 'GuionCHO.txt'}, page_content='De acuerdo a su consulta, esta solicitud se debe realizar por medio del correo electrónico de servicioalcliente@finanzauto.com.co adjuntando la siguiente documentación:\n\n* Carta donde nos informe el trámite a solicitar, es necesario indicar el motivo y los cambios deseados.\n* La solicitud debe tener firma, huella, número de cédula y número de celular legibles. \n* Fotocopia de la tarjeta de propiedad por ambas caras.\n* Certificado de la Dijin no mayor a un mes (en el c